In [ ]:
import numpy as np
import pandas as pd
import torch, torch.nn as nn

## Model Based Reinforcement Learning Agent

In the next cell, we define the random shooting Model Based Reinforcement Learning Agent, the SOTA method in HVAC control research.

In [ ]:
def model_predict_fn(state_vector, model_path):
    # use cuda if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # load the model
    model_demo = nn.Sequential(
        nn.Linear(8, 200),
        nn.SiLU(),
        nn.Linear(200, 200),
        nn.SiLU(),
        nn.Linear(200, 200),
        nn.SiLU(),
        nn.Linear(200, 1),
    )
    model_demo.load_state_dict(torch.load(model_path))
    model_demo.eval()
    prediction = model_demo(torch.tensor(state_vector.astype(np.float32))).detach().numpy()
    return prediction

def choose_action(state, disturbance, model_predict_fn, horizon, n_samples, environment_forecast, curr_step, model_path, winter=True):

    '''
    Safe rule-based policy.
    Override the MPC agent when required.
    '''
    if curr_step % 96 < 31 or curr_step % 96 > 84: # empty building
        return [30, 15]
    if curr_step % 96 == 31 or curr_step % 96 == 32: # pre-occupied
        return [22, 22]
    if state < 20.0 or state > 23.5: # violation
        return [22, 22]

    zone_temperature_matrix = np.zeros((n_samples, horizon+1, 1))
    # populate every n_samples at time 0 with the current zone temperature
    for i in range(n_samples):
        zone_temperature_matrix[i, 0] = state

    action_matrix = np.zeros((n_samples, horizon, 2))

    # populate the action matrix with random integers between 15 and 26 inclusive
    for i in range(n_samples):
        for j in range(horizon):
            action_matrix[i, j, 0] = np.random.randint(21, 30) # cooling
            action_matrix[i, j, 1] = np.random.randint(15, 23) # heating

    environment_forecast = environment_forecast[curr_step:curr_step+horizon]
    # replace the first row of environment_forecast with the disturbance
    environment_forecast.iloc[0] = disturbance

    for t in range(horizon):
        action_vector = action_matrix[:, t]
        zone_temperature_vector = zone_temperature_matrix[:, t]
        environment_forecast_vector = np.tile(environment_forecast.iloc[t], (n_samples, 1))

        state_vector = np.concatenate([environment_forecast_vector, zone_temperature_vector, action_vector], axis=1)

        next_zone_temperature_vector = model_predict_fn(state_vector, model_path)

        zone_temperature_matrix[:, t+1] = next_zone_temperature_vector
    
    zone_temperature_matrix = zone_temperature_matrix[:, 1:]
    
    rewards_vector = evaluate_rewards(zone_temperature_matrix, action_matrix, environment_forecast, winter)

    return action_matrix[np.argmax(rewards_vector), 0]


def evaluate_rewards(zone_temperature_matrix, action_matrix, env_forecast, winter):
    rewards_vector = np.zeros(zone_temperature_matrix.shape[0])
    for trajectory in range(zone_temperature_matrix.shape[0]):
        sum_reward = 0
        for t in range(zone_temperature_matrix.shape[1]):
            # get 'Zone People Occupant Count(SPACE1-1)' in the env_forcast
            people_count = env_forecast.iloc[t].at['Zone People Occupant Count(SPACE1-1)']
            zone_temp = zone_temperature_matrix[trajectory, t]
            action = action_matrix[trajectory, t]
            step_reward = 0
            if people_count > 0:
                if not winter:
                    if zone_temp < 23.0:
                        step_reward += zone_temp - 23.0
                    elif zone_temp > 26.0:
                        step_reward += 26.0 - zone_temp
                else:
                    if zone_temp < 20.0:
                        step_reward += zone_temp - 20.0
                    elif zone_temp > 23.5:
                        step_reward += 23.5 - zone_temp
                if not winter:
                    step_reward -= 0.01*abs(30-action[0]) # 30 means no cooling applied
                else:
                    step_reward -= 0.01*abs(action[1]-15) # 15 means no heating applied
            else:
                if not winter:
                    step_reward -= abs(30-action[0]) # 30 means no cooling applied
                else:
                    step_reward -= abs(action[1]-15) # 15 means no heating applied
            sum_reward += 0.99**t*step_reward
        if type(sum_reward) == np.ndarray:
            rewards_vector[trajectory] += sum_reward[0]
        elif type(sum_reward) == np.float64:
            rewards_vector[trajectory] += sum_reward
    return rewards_vector

## Generate Data

In the next few cells, we generate decision data by sampling the neighborhood of historical data.

In [ ]:
from data.data_manager import *

X, Y = data_tree['pittsburgh']['winter']

print(X)

In [ ]:
# add a column to X that is the time, represented using float 0 to 23.75
# the time is calculated as row_index mod 96 / 4
X['time'] = X.index % 96 / 4

print(X['time'])

In [ ]:
Historical_X = X.drop(columns=['time'])
# print the column names of Historical_X
print(Historical_X.columns)

In [ ]:
from datetime import datetime

def generate_decision_data(noise_level, samples, samples_per_input, city, model_path, save_path='Decision_data_drame.csv', winter=True):

    env_forecast = get_environment_forecast(city=city)

    for sample_idx in range(samples):
        if sample_idx % 30 == 0:
            print('Sample #{}'.format(sample_idx))

        df = []

        # sample a number from 0 to size of Historical_X - 21 that mod 96 > 31 and mod 96 < 84
        current_step = np.random.randint(0, Historical_X.shape[0]-21)
        while current_step % 96 < 31 or current_step % 96 > 84:
            current_step = np.random.randint(0, Historical_X.shape[0]-21)
        # get the row of Historical_X at index current_step
        input_x = Historical_X.iloc[current_step]

        zone_temp = input_x['Zone Air Temperature(SPACE1-1)']

        while zone_temp < 20.0 or zone_temp > 23.5:
            current_step = np.random.randint(0, Historical_X.shape[0]-21)
            while current_step % 96 < 31 or current_step % 96 > 84:
                current_step = np.random.randint(0, Historical_X.shape[0]-21)
            # get the row of Historical_X at index current_step
            input_x = Historical_X.iloc[current_step]
            zone_temp = input_x['Zone Air Temperature(SPACE1-1)']

        people_count = input_x['Zone People Occupant Count(SPACE1-1)']

        noised_input = input_x + np.random.normal(0, noise_level*np.std(input_x, axis=0), input_x.shape)
        # separate obs and disturbance
        obs = noised_input['Zone Air Temperature(SPACE1-1)']
        disturbance = noised_input.drop('Zone Air Temperature(SPACE1-1)')
        disturbance = disturbance.drop('Zone Thermostat Heating Setpoint Temperature(SPACE1-1)')
        disturbance = disturbance.drop('Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)')

        '''
        For any of the 'Site Outdoor Air Relative Humidity(Environment)',
       'Site Wind Speed(Environment)',
       'Site Direct Solar Radiation Rate per Area(Environment)',
        if the value is negative, set it to 0
        '''
        disturbance['Site Outdoor Air Relative Humidity(Environment)'] = max(0, disturbance['Site Outdoor Air Relative Humidity(Environment)'])
        disturbance['Site Wind Speed(Environment)'] = max(0, disturbance['Site Wind Speed(Environment)'])
        disturbance['Site Direct Solar Radiation Rate per Area(Environment)'] = max(0, disturbance['Site Direct Solar Radiation Rate per Area(Environment)'])
        
        # set 'Zone People Occupant Count(SPACE1-1)' to people_count
        disturbance['Zone People Occupant Count(SPACE1-1)'] = people_count

        for _ in range(samples_per_input):
            start_time = datetime.now()
            action = choose_action(obs, 
                                disturbance,
                                model_predict_fn, 
                                horizon=20, 
                                n_samples=1000, 
                                environment_forecast=env_forecast, 
                                curr_step=current_step, 
                                model_path=model_path, 
                                winter=winter)
            end_time = datetime.now()
            df.append([current_step % 96 / 4, 
                    disturbance['Site Outdoor Air Drybulb Temperature(Environment)'],
                    disturbance['Site Outdoor Air Relative Humidity(Environment)'],
                    disturbance['Site Wind Speed(Environment)'],
                    disturbance['Site Direct Solar Radiation Rate per Area(Environment)'],
                    disturbance['Zone People Occupant Count(SPACE1-1)'],
                    obs,
                    action[1],
                    (end_time-start_time).total_seconds()])
            # if _ % 10 == 0:
            #     print(action, ' :: ', current_step, ' :: #{}'.format(_), ' :: ', (end_time-start_time).total_seconds())

        # a csv file already exists at the save_path, append to it
        if os.path.exists(save_path):
            df_csv = pd.DataFrame(df, 
                            columns=['time',
                            'Site Outdoor Air Drybulb Temperature(Environment)',
                            'Site Outdoor Air Relative Humidity(Environment)',
                            'Site Wind Speed(Environment)',
                            'Site Direct Solar Radiation Rate per Area(Environment)',
                            'Zone People Occupant Count(SPACE1-1)',
                            'Zone Air Temperature(SPACE1-1)', 
                            'action',
                            'overhead'])
            df_csv.to_csv(save_path, mode='a', header=False)
        # a csv file does not exist at the save_path, create it
        else:
            df_csv = pd.DataFrame(df, 
                                columns=['time',
                                'Site Outdoor Air Drybulb Temperature(Environment)',
                                'Site Outdoor Air Relative Humidity(Environment)',
                                'Site Wind Speed(Environment)',
                                'Site Direct Solar Radiation Rate per Area(Environment)',
                                'Zone People Occupant Count(SPACE1-1)',
                                'Zone Air Temperature(SPACE1-1)', 
                                'action',
                                'overhead'])
            df_csv.to_csv(save_path)

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

# for _ in range(1000):
#     # for noise_level in [0.01, 0.03, 0.05, 0.07, 0.09]:
#     for noise_level in [0.01]:
#         # for model in [10, 30, 100, 300, 1200]:
#         for model in [1200]:
#             print('noise_level={}, model={}'.format(noise_level, model))
#             generate_decision_data(noise_level=noise_level,
#                                 samples=100,
#                                 samples_per_input=30,
#                                 city='pittsburgh',
#                                 model_path='zmodels/model_pittsburgh_winter_{}.pth'.format(model),
#                                 save_path='IP_decisions_2/IP_decisions_noise={}_model={}.csv'.format(noise_level, model),
#                                 winter=True
#                                 )

for _ in range(1000):
    # for noise_level in [0.01, 0.03, 0.05, 0.07, 0.09]:
    for noise_level in [0.01]:
        # for model in [10, 30, 100, 300, 1200]:
        for model in [1200]:
            # record time to calculate overhead
            t0 = datetime.now()
            print('noise_level={}, model={}'.format(noise_level, model))
            generate_decision_data(noise_level=noise_level,
                                samples=1,
                                samples_per_input=30,
                                city='tucson',
                                model_path='zmodels/model_tucson_winter_{}.pth'.format(model),
                                save_path='IP_decisions_2/IP_decisions_tucson_noise={}_model={}.csv'.format(noise_level, model),
                                winter=True
                                )
            t1 = datetime.now()
            print('Time elapsed: {}'.format((t1-t0).total_seconds()))